### SCONE setup

In [2]:
from sconetools import sconepy
sconepy.set_log_level(3)
print('SCONE Version', sconepy.version())
sconepy.set_array_dtype_float32()
# sconepy is in the /bin

sconepy found at /opt/scone/lib
11:53:01 Successfully initialized OpenSim3 version 3.3-2021-01-28
11:53:01 Loaded settings from /home/eddlai/.config/SCONE/scone-settings.zml
SCONE Version 2.3.1.2903


### model_info

In [7]:
model = sconepy.load_model('SCONE_data/H0918_osim3.scone')
print("type(model):", type(model)), print("model:", model)
for bod in model.bodies():
    # if bod.name().startswith('t'): # only items starting with 't' to avoid clutter
    print(f"body {bod.name()} mass={bod.mass():.3f} inertia={bod.inertia_diag()}")
for mus in model.muscles():
    # if mus.name().startswith('g'): # only items starting with 'g' to avoid clutter
    print(f"muscle {mus.name()} L={mus.fiber_length_norm():.3f} V={mus.fiber_velocity_norm():.3f} F={mus.force_norm():.3f}")

type(model): <class 'sconepy.Model'>
model: <sconepy.Model object at 0x7a855876fb70>
body ground mass=0.000 inertia=[ 0.000000 0.000000 0.000000 ]
body pelvis mass=11.777 inertia=[ 0.102800 0.087100 0.057900 ]
body femur_r mass=9.301 inertia=[ 0.133900 0.035100 0.141200 ]
body tibia_r mass=3.708 inertia=[ 0.050400 0.005100 0.051100 ]
body calcn_r mass=1.250 inertia=[ 0.001400 0.003900 0.004100 ]
body femur_l mass=9.301 inertia=[ 0.133900 0.035100 0.141200 ]
body tibia_l mass=3.708 inertia=[ 0.050400 0.005100 0.051100 ]
body calcn_l mass=1.250 inertia=[ 0.001400 0.003900 0.004100 ]
body torso mass=34.237 inertia=[ 1.474500 0.755500 1.431400 ]
muscle hamstrings_r L=1.306 V=0.000 F=0.137
muscle bifemsh_r L=0.891 V=0.000 F=0.009
muscle glut_max_r L=1.055 V=0.000 F=0.016
muscle iliopsoas_r L=0.547 V=0.000 F=0.002
muscle rect_fem_r L=0.620 V=0.000 F=0.005
muscle vasti_r L=1.106 V=0.000 F=0.028
muscle gastroc_r L=1.145 V=0.000 F=0.037
muscle soleus_r L=1.342 V=-0.000 F=0.171
muscle tib_ant_r 

In [59]:
import numpy as np
import random
import time
random_seed = 1
rng = np.random.default_rng(random_seed)

time_flag = []
time_flag_in_sim = []
max_time = 15.0
store_data = True
time_flag.append(time.perf_counter())

min_com_height=0.3

model = sconepy.load_model('data/edd_osim3.scone')
model.reset()
model.set_store_data(True)
muscle_activations = 0.1 + 0.4 * rng.random((len(model.muscles())))
model.init_muscle_activations(muscle_activations)
time_flag.append(time.perf_counter())

dof_positions = model.dof_position_array()
dof_positions += 0.1 * rng.random(len(dof_positions)) - 0.05
model.set_dof_positions(dof_positions)
time_flag.append(time.perf_counter())

# UPDATE others elements, such as muscles and states
model.init_state_from_dofs()
time_flag.append(time.perf_counter())
time_flag_in_sim.append(time.perf_counter())
# [START, loop1_muscle->actuator, loop2_internal loop]

for t in np.arange(0, max_time, 0.01):
    mus_in = model.muscle_force_array()
    mus_in += model.muscle_fiber_length_array() - 1
    mus_in += 0.2 * model.muscle_fiber_velocity_array()
    model.set_actuator_inputs(mus_in)
    time_flag_in_sim.append(time.perf_counter())

    # Since there are many internal iteration in the simulation,
    # So we need this command to update the model
    # the simulation step size is Varaiable
    # .scone set the accuracy and effect the step size
    model.advance_simulation_to(t)
    time_flag_in_sim.append(time.perf_counter())

    # the condition to abort the simulation
    com_y = model.com_pos().y
    # if com_y < min_com_height:
    #     print(f'Aborting simulation at t={model.time():.3f} com_y={com_y:.4f}')
    #     break

time_flag.append(time.perf_counter())

Aborting simulation at t=1.390 com_y=0.2934


In [60]:
import pandas as pd
time_flag_diffs = np.diff(time_flag)
time_flag_in_sim_diffs = np.diff(time_flag_in_sim)

titles_time_flag = [
    "1. Model Loading and Muscle Initialization Time",
    "2. Model DOF set",
    "3. init_state_from_dofs",
    "4. Model simulation"
]

df = pd.DataFrame({
    "Time Flag": time_flag[:-1], 
    "Time Flag Diffs": time_flag_diffs,
    "Titles": titles_time_flag
})

titles_time_flag_in_sim = ["1. Start Simulation"] + [
    f"{int(i/2+1)}_2. loop1_muscle->actuator" if i % 2 != 0 else f"{int(i/2)}_3. loop2_internal loop"
    for i in range(1, len(time_flag_in_sim_diffs))
]

df_sim = pd.DataFrame({
    "Time Flag In Sim": time_flag_in_sim[:-1],
    "Time Flag In Sim Diffs": time_flag_in_sim_diffs,
    "Titles": titles_time_flag_in_sim
})

In [61]:
if store_data:
    dirname = 'sconepy_example_' + model.name()
    filename = model.name() + f'_{random_seed}_{model.time():0.3f}_{model.com_pos().y:0.3f}'
    model.write_results(dirname, filename)
    print(f'SCONE/results/{dirname}/{filename}. Please use SCONE Studio to replay the .sto file.', flush=True)

SCONE/results/sconepy_example_H0918v3/H0918v3_1_1.390_0.293. Please use SCONE Studio to replay the .sto file.


In [62]:
import pandas as pd

sto_file = '/home/eddlai/SCONE/results/sconepy_example_H0918v3/H0918v3_1_14.990_-0.864.sto'
with open(sto_file, 'r') as file:
    lines = file.readlines()
header_end_idx = lines.index('endheader\n') + 1
data_lines = lines[header_end_idx:]
df = pd.read_csv(sto_file, delim_whitespace=True, skiprows=header_end_idx)
df.head(5)


/tmp/ipykernel_6487/830801426.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(sto_file, delim_whitespace=True, skiprows=header_end_idx)


,time,pelvis_tilt,pelvis_tx,pelvis_ty,hip_flexion_r,knee_angle_r,ankle_angle_r,hip_flexion_l,knee_angle_l,ankle_angle_l,...,leg1_r.grf_norm_z,leg1_r.grf_x,leg1_r.grf_y,leg1_r.grf_z,leg1_r.grm_x,leg1_r.grm_y,leg1_r.grm_z,leg1_r.cop_x,leg1_r.cop_y,leg1_r.cop_z
0,0.00,-0.640520,-0.023769,0.763318,1.37430,-1.22321,0.484406,1.44243,-1.19925,0.440455,...,0,-0.0000,-0.000,0,-0.0000,-0.00000,-0.0000,0.000000,0.000000e+00,0.00000
1,0.01,-0.640634,-0.023866,0.763117,1.37638,-1.22156,0.277686,1.44317,-1.19659,0.267916,...,0,-331.1140,341.970,0,-31.2629,-30.27050,23.8615,0.069777,-2.478960e-16,0.09142
2,0.02,-0.639076,-0.023256,0.762370,1.37002,-1.21541,0.301205,1.43279,-1.18041,0.247834,...,0,14.4127,164.162,0,-15.0077,1.31761,12.1363,0.073929,-2.626480e-16,0.09142
3,0.03,-0.636109,-0.022133,0.761119,1.36011,-1.20925,0.316244,1.41516,-1.15639,0.253510,...,0,12.4106,268.293,0,-24.5274,1.13458,19.6859,0.073375,-2.606790e-16,0.09142
4,0.04,-0.631997,-0.020690,0.759372,1.34975,-1.20514,0.333195,1.39596,-1.14059,0.291112,...,0,27.4844,349.574,0,-31.9581,2.51262,25.5432,0.073069,-2.595950e-16,0.09142
